In [ ]:
# pip install imgaug

In [1]:
import random
import numpy as np
import os
import shutil
import cv2
import imgaug as ia
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from PIL import Image
from glob import glob
from PIL import Image, ImageDraw, ImageFont, ImageOps
from tqdm import tqdm
import PIL.ImageOps
# imgaug의 시드 고정
ia.seed(42)
# directory로 이동
# %cd ..
# %cd yolov5/final_data_

In [2]:
pwd

'/home/tlrkrwlsmd/yolov5'

In [3]:
def route(t):
    file_path = os.path.join("/home/tlrkrwlsmd/yolov5/dataset/images", t)
    pth = []
    for f in os.listdir(file_path):
        pth.append(os.path.join(file_path, f))
    return pth
def annot_route(t):
    file_path = os.path.join("/home/tlrkrwlsmd/yolov5/dataset/labels", t)
    pth = []
    for f in os.listdir(file_path):
        pth.append(os.path.join(file_path, f))
    return pth
pth_tr = route("train")
annot_tr = annot_route("train")

In [4]:
len(pth_tr), len(annot_tr)

(1117, 1117)

### Data Augmentation 대상
- class imbalance를 해결하기 위해서 Augmentation을 진행한다.
- annotation 中 1번 클래스가 상당 수를 차지하는 case에 대해서 augmentation을 진행한다

### 총 3가지의 Augmentation을 구현
1. 180°회전(Rotation)
2. Noise 추가
3. 객체를 기준으로 한 Crop

In [ ]:
"""
Noise를 추가하는 함수
"""

def make_noise(std, img):
    height, width, ch = img.shape
    # 각 channel에 적용
    # 하나 더 추가
    # 빈 배열 형성
    img_noise = np.zeros((height, width, ch), dtype = np.float64)
    for i in range(height):
        for j in range(width):
            for k in range(ch):
                # gaussian distribution을 따르는 random 숫자를 넣는다
                mk_noise = np.random.normal()
                set_noise = std * mk_noise
                noise_input = img[i][j][k] + set_noise
                if noise_input > 255:
                    noise_input = 255
                if noise_input < 0:
                    noise_input = 255
                img_noise[i][j][k] = noise_input
    return img_noise

In [5]:
"""
객체를 기준으로 Crop하는 함수
"""
def make_crop(img_annot, image):
    with open(img_annot, "r") as f:
        data = f.readlines()
        # cv2는 height, width, channel 순이다.
        img = image
        height, width = img.shape[0], img.shape[1]
        # 각 line에서 xmin, xmax, ymin, ymax를 뽑아낸다.
        xmin = []
        ymin = []
        xmax = []
        ymax = []
        xcenter = []
        ycenter = []
        new_xcenter = []
        new_ycenter = []
        WIDTH = []
        w_l = []
        HEIGHT = []
        h_l = []
        class_no = []
        for line in data:
            info = line.split()
            x_center, y_center, w, h = float(info[1]), float(info[2]), float(info[3]), float(info[4])
            class_no.append(int(line[0]))
            xcenter.append(int(x_center * width))
            ycenter.append(int(y_center * height))
            xmin.append(int((x_center - w/2) * width))
            ymin.append(int((y_center - h/2) * height))
            xmax.append(int((x_center + w/2) * width))
            ymax.append(int((y_center + h/2) * height))
            WIDTH.append(int(w * width))
            HEIGHT.append(int(h * height))
        xmin, ymin, xmax, ymax = min(xmin), min(ymin), max(xmax), max(ymax)
        # crop width
        crop_width = xmax - xmin
        crop_height = ymax - ymin
        for wid in WIDTH:
            if wid/crop_width > 1:
                w_l.append(1.0)
            else:
                w_l.append(wid/crop_width)
        for hei in HEIGHT:
            if hei/crop_height > 1:
                h_l.append(1.0)
            else:
                h_l.append(hei/crop_height)
        for x in xcenter:
            if (x-xmin)/crop_width > 1:
                new_xcenter.append(1.0)
            else:
                new_xcenter.append((x - xmin)/crop_width)
        for y in ycenter:
            if (y-ymin)/crop_height > 1:
                new_ycenter.append(1.0)
            else:
                new_ycenter.append((y - ymin)/crop_height)
    return xmin, ymin, xmax, ymax, xcenter, ycenter, new_xcenter, new_ycenter, w_l, h_l, class_no

In [12]:
# 1. 사진을 rotation
# 2. noise 추가
# 3. CenterCrop을 진행
# num값의 조정을 통해서 선택적으로 증강을 진행할 수 있다.
def data_augmentation(num, img, annot, n):
    image = cv2.imread(img)
    img_pth = img
    img_annot = annot
    """
    180°회전하는 부분(num == 0)
    """
    if num == 0:
        # annotation file을 읽는다.
        f = open(img_annot)
        bbx = f.readlines()
        # normalize된 것을 복구하기 위한 작업
        d_w = image.shape[1] # 폭
        d_h = image.shape[0] # 높이
        # 하나씩 읽어온다.
        class_no = []
        for i in range(len(bbx)):
            bbxs = bbx[i].split(" ")
            for j in range(len(bbxs)):
                # 이는 class에 해당한다.
                if j == 0:
                    bbxs[j] = int(bbxs[j])
                    class_no.append(int(bbxs[j]))
                # str 형태를 float으로 바꿔온다
                else:
                    x = float(bbxs[j])
                    bbxs[j] = x
            # 대체하는 작업
            bbx[i] = bbxs
        # 작업해야할 bounding box
        bbx_new = []
        for i in range(len(bbx)):
            # 원래의 x_center
            x_center = bbx[i][1] * d_w
            # 원래의 y_center
            y_center = bbx[i][2] * d_h
            # 원래의 폭
            w = bbx[i][3] * d_w
            # 원래의 높이
            h = bbx[i][4] * d_h
            # x1, x2는 각각 boundingbox의 x좌표의 양 끝단
            # y1, y2는 각각 boundingbox의 y좌표의 양 끝단
            x1 = int(x_center - w/2)
            y1 = int(y_center - h/2)
            x2 = int(x_center + w/2)
            y2 = int(y_center + h/2)
            bbx_new.append([x1, y1, x2, y2])
        ia_bbxs = []
        for b in bbx_new:
            ia_bbxs.append(BoundingBox(x1 = b[0], y1 = b[1], x2 = b[2], y2 = b[3]))
        bound_box = BoundingBoxesOnImage(ia_bbxs, shape = image.shape)
        # 180도 회전을 다음과 같이 진행한다.
        seq = iaa.Sequential([iaa.Affine(rotate = 180)])
        # 180도 회전 적용
        img_aug, bbx_aug = seq(image = image, bounding_boxes = bound_box)
        # 이미지를 각 파일에 저장
        storage_dir = os.path.join(img_pth[:45], "rotation_"+str(n)+".png")
        cv2.imwrite(storage_dir, img_aug)
        # 정규화를 시켜서 저장한다.
        annot_file_path = os.path.join(img_annot[:45], "rotation_"+str(n)+".txt")
        with open(annot_file_path, "w") as f:
            for a in range(len(bbx_aug)):
                class_id = class_no[a]
                # opencv의 경우는 높이, 폭, 채널 순서
                dx = 1/int(image.shape[1])
                dy = 1/int(image.shape[0])
                x1 = bbx_aug[a][0][0]
                x2 = bbx_aug[a][1][0]
                y1 = bbx_aug[a][0][1]
                y2 = bbx_aug[a][1][1]
                x_center = (x1+x2)/2 * dx
                y_center = (y1+y2)/2 * dy
                w = (x2-x1) * dx
                h = (y2-y1) * dy
                f.write(f"{class_id} {x_center:.5f} {y_center:.5f} {w:.5f} {h:.5f}\n")
    # Noise 추가
    # 기본적으로 noise로는 Gaussian noise를 추가한다.
    # if num == 1:
    #     noise_img = make_noise(12, image)
    #     storage_dir = os.path.join(img_pth[:14], "noise_"+str(n)+".png")
    #     # 이미지 저장
    #     cv2.imwrite(storage_dir, noise_img)
    #     # annotation 저장
    #     new_annot_file_path = os.path.join(img_annot[:14], "noise_"+str(n)+".txt")
    #     original_annot_path = img_annot
    #     shutil.copyfile(original_annot_path, new_annot_file_path)
    # crop 진행.
    if num == 1:
        # Center 값만 변형 시킨다.
        # crop을 한다고 해서 Width와 height가 변하는 것은 아니다
        # 시작 기준점이 바뀐다.
        mk_annot = make_crop(img_annot, image)
        xmin, xmax, ymin, ymax = mk_annot[0], mk_annot[2], mk_annot[1], mk_annot[3]
        cropped_image = image[ymin:ymax, xmin:xmax]
        storage_dir = os.path.join(img_pth[:45], "cropped_"+str(n)+".png")
        cv2.imwrite(storage_dir, cropped_image)
        # annotation을 적어야 한다.
        annot_file_path = os.path.join(img_annot[:45], "cropped_"+str(n)+".txt")
        # new_xcenter, new_ycenter
        Xcenter = mk_annot[6]
        Ycenter = mk_annot[7]
        width_list = mk_annot[8]
        height_list = mk_annot[9]
        class_no = mk_annot[10]
        with open(annot_file_path, "w") as f:
            for idx in range(len(class_no)):
                class_id = class_no[idx]
                x_center = Xcenter[idx]
                y_center = Ycenter[idx]
                w = width_list[idx]
                h = height_list[idx]
                f.write(f"{class_id} {x_center:.5f} {y_center:.5f} {w:.5f} {h:.5f}\n")

In [13]:
for n in tqdm(range(len(annot_tr))):
    annot = annot_tr[n]
    imgs = pth_tr[n]
    # num이 0, 1, 2 순서대로 rotation, Gaussian Noise, Crop
    for num in tqdm(range(2)):
        data_augmentation(num, imgs, annot, n)

  0%|                                                     | 0/2 [00:00<?, ?it/s]Premature end of JPEG file

 50%|██████████████████████▌                      | 1/2 [00:00<00:00,  1.73it/s]Premature end of JPEG file

  0%|                                                     | 0/2 [00:00<?, ?it/s]Premature end of JPEG file

 50%|██████████████████████▌                      | 1/2 [00:00<00:00,  1.72it/s]Premature end of JPEG file

  0%|                                                     | 0/2 [00:00<?, ?it/s]Premature end of JPEG file

 50%|██████████████████████▌                      | 1/2 [00:00<00:00,  1.74it/s]Premature end of JPEG file

  0%|                                                     | 0/2 [00:00<?, ?it/s]Premature end of JPEG file

 50%|██████████████████████▌                      | 1/2 [00:00<00:00,  1.61it/s]Premature end of JPEG file

  0%|                                                     | 0/2 [00:00<?, ?it/s]Premature end of JPEG file

 50%|██████████████████████▌

In [11]:
pth_tr[0][:45]

'/home/tlrkrwlsmd/yolov5/dataset/images/train/'

In [ ]:
img_name = [i[:-4] for i in pth_tr]
img_names = [i[45:] for i in img_name]
annot_name = [i[:-4] for i in annot_tr]
annot_names = [i[45:] for i in annot_name]
for i in range(len(img_names)):
    if img_names[i] not in annot_names:
        print(i)

In [ ]:
pth_tr[1117]

In [ ]:
dir_main = "./yolov5/final_data_/"
filenames_image = glob(f"{dir_main}/images/train/*.png")[:][85:]
filenames_label = [filename.replace('images', 'labels').replace('png', 'txt') for filename in filenames_image]
classes = ["belt", "no belt", "hard", "no hard", "helmet", "no helmet"]

print(len(filenames_image), len(filenames_label))
color = []
# color 생성하는 부분
for _ in range(6):
    c = list(np.random.choice(range(256), size=3)) + [255]
    c = tuple(c)
    color.append(c)

# annotation 정보가 제대로 저장됐는지 확인하기 위한 bounding box를 시각화하는 함수
def draw_bbox(draw, bbox, label, color=(0, 255, 0, 255), confs=None, size=15):
    font = ImageFont.truetype("arial.ttf", size=size)
    draw.rectangle(bbox, outline=color, width =3)
    def set_alpha(color, value):
        background = list(color)
        background[3] = value
        return tuple(background)
    background = set_alpha(color, 50)
    draw.rectangle(bbox, outline=color, fill=background, width =3)
    background = set_alpha(color, 150)
    text = f"{label}" + ("" if confs==None else f":{conf:0.4}")
    text_bbox = bbox[0], bbox[1], bbox[0]+len(text)*10, bbox[1]+25
    draw.rectangle(text_bbox, outline=color, fill=background, width =3)
    draw.text((bbox[0]+5, bbox[1]+5), text, (0,0,0), font=font)

cnt = 1
for filename_image, filename_label in tqdm(zip(filenames_image, filenames_label)):
    img = Image.open(filename_image)
    img = img.resize((640, 640))
    width, height = img.size
    draw = ImageDraw.Draw(img, 'RGBA')
    with open(filename_label, 'r') as f:
        labels = f.readlines()
        # strip: 개행 문자 제거.
        labels = list(map(lambda s: s.strip().split(), labels))
    # bounding box를 그리는 부분
    for label in labels:
        # class는 0부터 시작!
        cls = int(label[0])
        x, y, w, h = map(float, label[1:])
        x1, x2 = width * (x-w/2), width * (x+w/2)
        y1, y2 = height * (y-h/2), height * (y+h/2)
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        draw_bbox(draw, bbox=(x1, y1, x2, y2), label=classes[cls], color=color[cls], size=15)
    img.save("{}".format(filename_image))
    print("{}".format(filename_image))
    img.show()

    cnt -= 1
    if cnt ==0:
        break

In [ ]:
"""
해당 부분은 상대적으로 적은 숫자의 class에 대해서 augmentation을 시키고 난 후에도 상대적인 비율이 여전히 해소되지 않는 경우
label 정보로 적은 클래스의 경우만 담는 과정이다.
"""


# label_train = "./labels/train"

# def change_label(mode):
#     label = label_train
#     under = os.listdir(label)
#     for und in under:
#         annot_pth = os.path.join(label, und)
#         if und[:8] == "rotation": 
#             with open(annot_pth, "r") as f:
#                 data = f.readlines()
#             with open(annot_pth, "w") as f:
#                 for i in data:
#                     ann = i.split()
#                     if int(ann[0]) == 1:
#                         class_id, x_center, y_center, w, h = ann[0], float(ann[1]), float(ann[2]), float(ann[3]), float(ann[4])
#                         f.write(f"{class_id} {x_center:.5f} {y_center:.5f} {w:.5f} {h:.5f}\n")

# change_label("train")
# # change_label("valid")

In [30]:
import os
import cv2
import numpy as np
import random
from tqdm import tqdm
import shutil
# 밝기 관련
def adjust_brightness(image, factor_range=(0.7, 1.3)):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    factor = random.uniform(factor_range[0], factor_range[1])
    hsv[:, :, 2] = np.clip(hsv[:, :, 2] * factor, 0, 255)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

# saturation
def adjust_saturation(image, factor_range=(0.7, 1.3)):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    factor = random.uniform(factor_range[0], factor_range[1])
    hsv[:, :, 1] = np.clip(hsv[:, :, 1] * factor, 0, 255)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

# 대조
def adjust_contrast(image, factor_range=(0.7, 1.3)):
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    factor = random.uniform(factor_range[0], factor_range[1])
    lab[:, :, 0] = np.clip(lab[:, :, 0] * factor, 0, 255)
    return cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

def apply_augmentation(image):
    # 명도, 채도, 대비 차례대로 적용
    brightened_image = adjust_brightness(image)
    saturated_image = adjust_saturation(brightened_image)
    contrasted_image = adjust_contrast(saturated_image)

    return contrasted_image

# 원본 이미지 파일들이 있는 폴더 경로
input_folder = '/home/tlrkrwlsmd/yolov5/dataset/images/train'
annot_folder = "/home/tlrkrwlsmd/yolov5/dataset/labels/train"

# 증강된 이미지들을 저장할 폴더 경로
output_folder = '/home/tlrkrwlsmd/yolov5/dataset/images/train'
out_folder = '/home/tlrkrwlsmd/yolov5/dataset/labels/train'

# 원본 이미지 파일들의 수를 세기
# num_original_images = sum(1 for _ in os.listdir(input_folder) if _.endswith('.jpg'))
original_images = [os.path.join('/home/tlrkrwlsmd/yolov5/dataset/images/train',  i) for i in os.listdir(input_folder) if i[-3:] == "jpg"]
annot_files = [os.path.join("/home/tlrkrwlsmd/yolov5/dataset/labels/train", i) for i in os.listdir(annot_folder) if i[0] == 'b']
print(len(annot_files))
print(f'현재 원본 파일 개수: {len(original_images)}')

# 사용자로부터 어그멘테이션 횟수 입력 받기
# num_augmentations = int(input('각 원본 파일에 대해 몇회차 어그멘테이션을 진행하시겠습니까? (원본 파일 개수와 관계없이 입력 가능): '))

# # 전체 랜덤 적용 횟수
# num_total_augmentations = num_original_images * num_augmentations

# # 각 원본 파일에 대해 랜덤 어그멘테이션을 적용하고 저장
for p in tqdm(range(len(original_images))):
    pth = original_images[p]
    image = cv2.imread(pth)
    if image is None:
        print("Error: Unable to read the image at path")
        continue
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # 어그멘테이션 적용
    augmented_image = apply_augmentation(image)
    # 저장할 파일명 설정
    output_filename = os.path.join("/home/tlrkrwlsmd/yolov5/dataset/images/train", "augment_"+str(p)+".png")
    annot_filename = os.path.join("/home/tlrkrwlsmd/yolov5/dataset/labels/train", "augment_"+str(p)+".txt")
    # 이미지 저장
    cv2.imwrite(output_filename, augmented_image)
    original_annot_path = annot_files[p]
    shutil.copyfile(original_annot_path, annot_filename)

# # print(f'{num_total_augmentations}개 증강 이미지 생성')

1117
현재 원본 파일 개수: 1117


100%|███████████████████████████████████████████████████████████████████████████████████████████| 1117/1117 [33:28<00:00,  1.80s/it]


In [14]:
augmented_image.shape

(3024, 4032, 3)